# Import Libraries

In [1]:
# EDA - Data Cleaning
import pandas as pd
import numpy as np
import missingno
from collections import Counter
import math

# EDA - Data Visualizations
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
from pylab import rcParams
%matplotlib inline
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot
plt.style.use('seaborn') # pretty matplotlib plots
plt.rc('font', size=14)
plt.rc('figure', titlesize=18)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=18)

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# logistic regression model
import statsmodels.api as sm 

pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)
pd.set_option('display.width', 1000)

pd.options.display.float_format = '{:.2f}'.format

# Import re module to use regular expression
import re

# Defined Functions

In [2]:
def nulls_review(df, df2):
    print('fund shape before: ', df.shape)
    nulls = pd.DataFrame(df2.isnull().sum().sort_values(ascending = False), columns=['Amount'])
    nulls = nulls.loc[nulls['Amount'] == len(df2.index)]
    nulls.index.name='Columns With Missing Values'
    print('Number of columns with 100% null values = ', len(nulls.index))
    drop_cols = list(nulls.index)
    df.drop(columns=drop_cols, inplace=True)
    print('fund shape after: ', df.shape)
    
# Output any correlation values over 0.65 - isolating high-correlated variables to remove from the analysis
def high_corr_and_check(X, threshold):
    corr_matrix = X.corr().abs()
    sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), 
                                     k=1).astype(np.bool))
                      .stack()
                      .sort_values(ascending=False))
    for index, value in sol.items():
        if value > threshold:
            print(index,value)
            
# Declare the filter function
def Filter(datalist, filt):
    # Search data based on regular expression in the list
    return [val for val in datalist
        if re.search(r''+filt, val)]


def show_values(axs, orient="v", space=.01):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.01)
                value = '{:.0f}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.5)
                value = '{:.0f}'.format(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax)
    else:
        _single(axs)

In [23]:
#Load in SCA Filings and Settlements Spreadsheet
df_pred = pd.read_csv('../01_data/03_final/capstone_modeling_final.csv', index_col='gvkey')
df_orig = pd.read_csv('../01_data/02_modified/company_data_cleaned.csv', index_col='gvkey')

# Isolate only the sample of companies used in my analysis
df = df_orig.join(df_pred)
# df = df.loc[df['suit_pred'].notnull()]


df['suitflag'] = df['suitflag'].astype(str)
# df['stko'] = df['stko'].astype(str)

In [20]:
df_orig

,tic,conm,GIC_Industry,GIC_SubIndustry,suitflag,idbflag,restatementflag,Vol_capx_Variance,Vol_cogs_Variance,Vol_cshfd_Variance,Vol_dltt_Variance,Vol_dp_Variance,Vol_emp_Variance,Vol_epspi_Variance,Vol_ni_Variance,Vol_ppent_Variance,Vol_sale_Variance,Vol_teq_Variance,Vol_txt_Variance,Vol_wcap_Variance,Vol_xint_Variance,Vol_xsga_Variance,Vol_roa_Variance,Vol_roe_Variance,STD_at,StdDev_capx,StdDev_cogs,StdDev_cshfd,StdDev_dltt,StdDev_dp,StdDev_emp,StdDev_epspi,StdDev_ni,StdDev_ppent,StdDev_sale,StdDev_teq,StdDev_txt,StdDev_wcap,STD_xido,StdDev_xint,StdDev_xsga,StdDev_roa,StdDev_roe
gvkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10008,SXI,STANDEX INTERNATIONAL CORP,Machinery,Industrial Machinary,No,D,1,-0.13,-43.20,0.00,0.00,-0.89,-4.40,-0.01,0.00,-93.23,-52.38,0.00,1.07,0.00,0.00,-11.61,0.00,0.00,26.33,4.47,31.46,0.05,22.28,0.95,0.26,0.56,7.15,6.28,50.24,40.43,2.12,21.06,7.94,0.69,8.37,0.01,0.01
10016,SWK,STANLEY BLACK & DECKER INC,Machinery,Industrial Machinary,No,D,1,0.00,-5062.00,0.00,0.00,-348.70,-0.39,0.00,-0.20,-1250.90,-999.90,0.00,-38.90,-2073.40,-0.20,-2086.50,-0.00,-0.00,572.43,90.24,738.29,8.89,415.92,38.61,5.77,1.71,290.82,135.86,1112.00,165.03,21.51,582.18,224.89,21.03,208.69,0.02,0.04
10030,SCX,STARRETT (L.S.) CO -CL A,Machinery,Industrial Machinary,No,D,1,0.00,4.63,0.00,0.00,0.00,0.00,-0.75,-5.04,0.00,0.00,-0.68,-2.18,0.00,-0.50,2.59,-0.03,-0.04,21.36,1.76,17.38,0.06,15.22,0.67,0.10,0.62,4.14,2.51,24.15,12.42,3.13,15.18,0.00,0.30,7.44,0.02,0.03
1004,AIR,AAR CORP,Aeorspace & Defense,Aerospace & Defense,No,D,1,0.00,25.33,0.00,0.00,0.00,-6.70,0.00,0.00,-5.22,29.33,0.00,-1.78,-16.24,-0.00,9.10,0.00,0.00,336.77,45.47,296.49,1.35,181.67,29.80,0.38,0.27,11.77,52.12,367.01,72.52,6.01,63.72,1.66,6.69,23.82,0.01,0.01
100424,JMHLY,JARDINE MATHESON HLDGS LTD,Industrial Congolomerates,Industrial Conglomerates,No,B,1,0.00,0.00,0.00,0.00,0.00,-390.00,-0.05,-17.00,0.00,0.00,-17.00,-873.00,4276.00,0.00,-5113.00,-0.00,-0.00,7342.51,450.68,3747.73,3.87,1038.80,128.41,38.73,2.69,959.52,3618.27,4411.95,5012.01,18.52,2670.21,0.00,19.02,597.19,0.02,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9325,SLI,SL INDUSTRIES INC,Electrical Equipment,Electrical Components & Equipment,No,D,1,0.00,0.00,0.00,0.00,0.00,-1.60,0.00,0.00,0.00,0.00,0.00,0.00,0.91,0.00,0.00,0.00,0.00,3.83,0.66,6.30,0.74,0.09,0.26,0.12,0.72,2.77,0.79,9.41,5.54,1.12,7.73,2.86,0.11,1.23,0.02,0.05
9619,SVT,SERVOTRONICS INC,Electrical Equipment,Electrical Components & Equipment,No,D,1,-0.49,-22.90,-2.11,-3.47,-0.66,-0.29,-1.08,-2.13,-6.16,-31.66,-21.31,-0.92,-18.84,-0.07,-5.00,-0.07,-0.10,0.84,0.48,1.30,0.07,1.58,0.03,0.02,0.54,1.05,0.41,1.78,1.08,0.28,1.53,0.66,0.01,0.31,0.04,0.05
9698,SIF,SIFCO INDUSTRIES,Aeorspace & Defense,Aerospace & Defense,No,D,1,-6.75,-78.45,-5.34,-1.19,-4.39,-0.36,-1.41,-7.45,-27.56,-107.36,-53.41,-3.79,-35.63,-0.13,-13.58,-0.09,-0.14,18.58,1.67,13.13,0.04,9.01,2.07,0.13,0.38,2.07,4.92,17.97,10.34,0.59,4.82,0.24,0.19,2.00,0.02,0.02


In [22]:
df

,tic,conm,GIC_Industry,GIC_SubIndustry,suitflag,idbflag,restatementflag,Vol_capx_Variance,Vol_cogs_Variance,Vol_cshfd_Variance,Vol_dltt_Variance,Vol_dp_Variance,Vol_emp_Variance,Vol_epspi_Variance,Vol_ni_Variance,Vol_ppent_Variance,Vol_sale_Variance,Vol_teq_Variance,Vol_txt_Variance,Vol_wcap_Variance,Vol_xint_Variance,Vol_xsga_Variance,Vol_roa_Variance,Vol_roe_Variance,STD_at,StdDev_capx,StdDev_cogs,StdDev_cshfd,StdDev_dltt,StdDev_dp,StdDev_emp,StdDev_epspi,StdDev_ni,StdDev_ppent,StdDev_sale,StdDev_teq,StdDev_txt,StdDev_wcap,STD_xido,StdDev_xint,StdDev_xsga,StdDev_roa,StdDev_roe,suit_pred
gvkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10008,SXI,STANDEX INTERNATIONAL CORP,Machinery,Industrial Machinary,No,D,1,-0.13,-43.20,0.00,0.00,-0.89,-4.40,-0.01,0.00,-93.23,-52.38,0.00,1.07,0.00,0.00,-11.61,0.00,0.00,26.33,4.47,31.46,0.05,22.28,0.95,0.26,0.56,7.15,6.28,50.24,40.43,2.12,21.06,7.94,0.69,8.37,0.01,0.01,0.07
10016,SWK,STANLEY BLACK & DECKER INC,Machinery,Industrial Machinary,No,D,1,0.00,-5062.00,0.00,0.00,-348.70,-0.39,0.00,-0.20,-1250.90,-999.90,0.00,-38.90,-2073.40,-0.20,-2086.50,-0.00,-0.00,572.43,90.24,738.29,8.89,415.92,38.61,5.77,1.71,290.82,135.86,1112.00,165.03,21.51,582.18,224.89,21.03,208.69,0.02,0.04,0.33
10030,SCX,STARRETT (L.S.) CO -CL A,Machinery,Industrial Machinary,No,D,1,0.00,4.63,0.00,0.00,0.00,0.00,-0.75,-5.04,0.00,0.00,-0.68,-2.18,0.00,-0.50,2.59,-0.03,-0.04,21.36,1.76,17.38,0.06,15.22,0.67,0.10,0.62,4.14,2.51,24.15,12.42,3.13,15.18,0.00,0.30,7.44,0.02,0.03,0.09
1004,AIR,AAR CORP,Aeorspace & Defense,Aerospace & Defense,No,D,1,0.00,25.33,0.00,0.00,0.00,-6.70,0.00,0.00,-5.22,29.33,0.00,-1.78,-16.24,-0.00,9.10,0.00,0.00,336.77,45.47,296.49,1.35,181.67,29.80,0.38,0.27,11.77,52.12,367.01,72.52,6.01,63.72,1.66,6.69,23.82,0.01,0.01,0.23
100424,JMHLY,JARDINE MATHESON HLDGS LTD,Industrial Congolomerates,Industrial Conglomerates,No,B,1,0.00,0.00,0.00,0.00,0.00,-390.00,-0.05,-17.00,0.00,0.00,-17.00,-873.00,4276.00,0.00,-5113.00,-0.00,-0.00,7342.51,450.68,3747.73,3.87,1038.80,128.41,38.73,2.69,959.52,3618.27,4411.95,5012.01,18.52,2670.21,0.00,19.02,597.19,0.02,0.03,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9325,SLI,SL INDUSTRIES INC,Electrical Equipment,Electrical Components & Equipment,No,D,1,0.00,0.00,0.00,0.00,0.00,-1.60,0.00,0.00,0.00,0.00,0.00,0.00,0.91,0.00,0.00,0.00,0.00,3.83,0.66,6.30,0.74,0.09,0.26,0.12,0.72,2.77,0.79,9.41,5.54,1.12,7.73,2.86,0.11,1.23,0.02,0.05,0.18
9619,SVT,SERVOTRONICS INC,Electrical Equipment,Electrical Components & Equipment,No,D,1,-0.49,-22.90,-2.11,-3.47,-0.66,-0.29,-1.08,-2.13,-6.16,-31.66,-21.31,-0.92,-18.84,-0.07,-5.00,-0.07,-0.10,0.84,0.48,1.30,0.07,1.58,0.03,0.02,0.54,1.05,0.41,1.78,1.08,0.28,1.53,0.66,0.01,0.31,0.04,0.05,0.46
9698,SIF,SIFCO INDUSTRIES,Aeorspace & Defense,Aerospace & Defense,No,D,1,-6.75,-78.45,-5.34,-1.19,-4.39,-0.36,-1.41,-7.45,-27.56,-107.36,-53.41,-3.79,-35.63,-0.13,-13.58,-0.09,-0.14,18.58,1.67,13.13,0.04,9.01,2.07,0.13,0.38,2.07,4.92,17.97,10.34,0.59,4.82,0.24,0.19,2.00,0.02,0.02,0.08


In [ ]:
'Vol_sale_Variance', 'StdDev_capx', 'StdDev_txt', 'GIC_SubIndustry'

In [ ]:
df[df['tic']=='CR']

In [ ]:
high_risk_companies = df[df['Prediction_Score'] > 50]

In [ ]:
plt.figure(figsize = (15, 10))
sns.set(font_scale = 2)

ax = sns.countplot(x='GIC_SubIndustry', data=df, hue='suitflag', saturation=1)
plt.legend(loc='upper right', labels=['No', 'Yes'], title='Suit Filed?')
plt.title('Number of Companies Where a Suit Was Filed by Stock Ownership Code')
plt.xticks(rotation=45, horizontalalignment='right')
show_values(ax)

plt.ylabel('Company Count')
plt.xlabel('GIC SubIndustries')
        
plt.show()

In [ ]:
high_risk_companies = df.loc[df['Prediction_Score'] >= 40].sort_values('Prediction_Score')
high_risk_companies[['tic', 'conm', 'Prediction_Score', 'Vol_sale_Variance_x', 'StdDev_capx_x', 'StdDev_txt_x', 'GIC_SubIndustry']]

In [ ]:
df[df['tic']=='CR']

In [ ]:
ax = sns.displot(df, x='Vol_sale_Variance_x', hue='suitflag', kind='kde', height=6, aspect=4, fill=True, palette='dark')
plt.xlim(-10000, 5000)

In [ ]:
# fig, ax = plt.subplots()
ax = sns.displot(df, x='StdDev_capx_x', hue='suitflag', kind='kde', height=6, aspect=4, fill=True, palette='dark')
plt.xlim(-500, 500)

In [ ]:
ax = sns.displot(df, x='StdDev_txt_x', hue='suitflag', kind='kde', height=6, aspect=4, fill=True, palette='dark')
plt.xlim(-500, 500)

In [ ]:
df[(df['suitflag']=='Yes') & (df['SettlementAmount'].notnull())][['gvkey','SettlementAmount']].sort_values('SettlementAmount').set_index('gvkey')

In [ ]:
df[(df['suitflag']=='Yes') & (df['SettlementAmount'].notnull())][['gvkey','SettlementAmount']].sort_values('SettlementAmount').set_index('gvkey').describe()

In [ ]:
df[df['tic']=='CR']